In [ ]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt
from cycler import cycler

%matplotlib inline  

## Work with data bundled in the Project
First, let's load a small Excel spreadsheet containing "Income Per Person" by country. "Income per person" is computed as the Gross Domestic Product (GDP) per person, adjusted for differences in purchasing power. Details on this particular dataset can be found [here](https://www.gapminder.org/data/documentation/gd001/).

Note, the spreadsheet `income_per_person_gdppercapita_ppp_inflation_adjusted.xlsx` has been bundled in this project directly. It is versioned for you automatically using git-lfs, and any changes will be captured. It's available in the `input` section of the project. 

Here, we use relative paths to help with portability of the project.

In [ ]:
income_file = '../input/income_per_person_gdppercapita_ppp_inflation_adjusted.xlsx'
income_dataframe = pd.read_excel(open(income_file, 'rb'), sheet_name='income_per_person_gdppercapita_')

Next, pull out the income per person from 1950 to 2018, sorted by 2018, and look at the top 15 countries

In [ ]:
income_dataframe = income_dataframe.sort_values(by=[2018], ascending=False)
num_rows = len(income_dataframe.index)
income_dataframe = income_dataframe.drop(range(1800,1950),axis=1)
income_dataframe = income_dataframe.drop(range(2019,2041),axis=1)
income_dataframe[:15]

## Plot some data and explore automatic versioning

Using a library like matplotlib, we can create a figure. This is automatically captured by the Gigantum Client and extracted for reference later.

Run this cell below and then switch back to the Gigantum Client tab in your browser. In the "Activity" section, you should see a new version created and the figure below captured!

Try changing `number_to_plot` to 5 and re-run the cell. You'll see the figure is captured again.

In [ ]:
number_to_plot = 10

fig = plt.figure(figsize=(10,10))
ax = plt.gca()
palette = plt.get_cmap('tab10')

for idx in range(0, number_to_plot):
    row = income_dataframe.iloc[idx]    
    country = row['country']
    row = row.drop('country')
    row.plot(marker='', color=palette(idx), linewidth=2, label=country, ax=ax)

plt.xlabel("Year")
plt.ylabel("Income")
plt.title("Income per person for the Top 10 countries in 2018")
plt.legend(loc=1, ncol=2);
xmin, xmax, ymin, ymax = plt.axis();

Automatically versioning is great, but sometimes you don't want to save a figure because it is sensitive or just not important. You can add special comments to a cell and the Client will change how activity is captured:

- `gtm:ignore` - The client will create a version, but NO activity will be captured
- `gtm:hide` - The client will create a version and capture activity, but it will be considered "unimportant" and displayed collapsed
- `gtm:show` - The client will create a version and capture activity, and it will be considered "important" and displayed expanded

Let's try another figure, but ignore it! If you check the Activity feed after running this cell, you'll see a new version, but no figure is saved.

In [ ]:
#gtm:ignore
fig = plt.figure(figsize=(10,10))
ax = plt.gca()
cyc = cycler(color='bgrcmykbgrcmykbgrcmykbgrcmykbgrcmykbgrcmyk')

for idx, cycle_args in zip(range(0, 100), cyc):
    row = income_dataframe.iloc[idx]
    country = row['country']
    row = row.drop('country')
    row.plot(marker='', linewidth=2, label=country, ax=ax, **cycle_args)

plt.xlabel("Year")
plt.ylabel("Income")
plt.title("A bad plot we probably should fix later...but let's just ignore for now!")
plt.legend(loc=1, ncol=2);
xmin, xmax, ymin, ymax = plt.axis();

## Accessing more data through Gigantum Datasets
Sometimes data gets large or worth sharing on its own. These are good scenarios to use Gigantum Datasets. This project has a Dataset that contains a large large CSV file (it's about 195MB) that contains the results from Stack Overflow's 2018 Developer Survey. 

If you simply opened this notebook and ran all cells to this point, you'll notice that data is missing! This is because Gigantum Datasets let you choose when you download data and what you download:

- Navigate back over the the Gigantum Client tab
- Go to the "Input" section
- Click on the "Download All" button (looks like an arrow pointing down) or open the dataset and download the `survey_results_public.csv` file.

Once this is complete, you should now be able to see the data in the Project. Dataset files get linked into Projects at container start time. You can see that the Dataset's files are available in a folder with the same name as the dataset in the `input` section.

In [ ]:
stackoverflow_file = '../input/stack-overflow-dev-survey-18/survey_results_public.csv'
result_data = pd.read_csv(open(stackoverflow_file, 'rt'));

With both files loaded, let's combine the two datasets into an interesting figure. First, pull out the salary data for all users while dropping any data that is missing or 0. The `ConvertedSalary` field takes care of normalizing responses to US dollars and 40 hour work week year round.

In [ ]:
result_data = result_data.dropna(subset=['Salary', 'SalaryType', 'ConvertedSalary'])
result_data = result_data[(result_data != 0).all(1)]
result_data = result_data[['Country', 'Salary','SalaryType', 'ConvertedSalary']]

Next, group by the number of responses to the survey by each country and grab the top 25 countries.

In [ ]:
response_counts = result_data.groupby('Country').count()
response_counts = response_counts.sort_values(by=['ConvertedSalary'], ascending=False)
response_counts = response_counts[:25]

Group the data again by Country and compute the median salary

In [ ]:
median_salary = result_data.groupby('Country').median()

Finally, combine the Income per person for each country with the median developer salary from the survey and number of responses.

In [ ]:
country_median_dev_by_gdppp = list()
income_file = '../input/income_per_person_gdppercapita_ppp_inflation_adjusted.xlsx'
df = pd.read_excel(open(income_file, 'rb'), sheet_name='income_per_person_gdppercapita_')

salary_data = list()
income_data = list()
country_data = list()
response_count_data = list()
for idx in range(0,25):
    country = response_counts.index[idx]
    salary = median_salary.loc[country]['ConvertedSalary']
    count = response_counts.loc[country][0]
    
    if country == "Russian Federation":
        country = "Russia"
        
    gdppp = df.loc[df['country'] == country, 2018]
    if not gdppp.empty:
        gdppp = gdppp.iloc[0]
    else:
        gdppp = None
        
    response_count_data
    salary_data.append(salary)
    income_data.append(gdppp)
    country_data.append(country)
    response_count_data.append(count)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.gca()
plt.scatter(income_data, salary_data, s=response_count_data, c=numpy.random.rand(25), alpha=.75, cmap=plt.get_cmap('hsv'));

for idx in range(0, 25):
    ax.annotate(country_data[idx], (income_data[idx], salary_data[idx]))
    
ax.patch.set_facecolor('lightgray')

plt.xlabel("Income Per Person")
plt.ylabel("Median Developer Salary")
plt.title("Median Developer Salary vs. Income Per Person in 2018 for top 25 responders");

As you create results that are worth keeping, you can write files to the "output" section of the Project. These changes will get versioned along with everything else. Be careful though, because writing too much data can bloat your project! For very large files, things you don't care about, or intermediate data products that are easily recomputed, you can write to the `untracked` directory in the "output" section. Files here are not versioned or synced

In [ ]:
output_file = "../output/untracked/original_figure.png"

# save the figure in the untracked directory so it isn't versioned or synced
fig.savefig(output_file, bbox_inches='tight');

This plot is decent, but probably good be made better! Gigantum Client manages branches for you, so you can go off on a tangent and try things out without losing your work. A branch keeps everything verisoned together, including your code, data, and environment configuration. Try switching to the "improved-plot" branch to see some improved plotting using seaborn. To switch:

- Save any changes you want to keep in this notebook and close the JupyterLab tab
- Go back to the Gigantum Client tab
- Stop the container by clicking on the "Stop" button (the toggle in the top left corner)
- Click on the Branch dropdown and select "improved-colors"
- Click on "Launch JupyterLab" and jump into the updated code!